<a href="https://colab.research.google.com/github/Bobisreallyme/TestRepo/blob/main/Cleaned_Embedding_Generation_06122024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install odfpy

In [ ]:
import os
import sqlite3
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
from transformers import AutoTokenizer, AutoModel

In [ ]:
model_checkpoint = r'/content/drive/MyDrive/ModelTrained/ModelandTokenizer10/Model_from_concatentation_4_26_2024_10'
streams_of_interest = r'YourStreamsOfInterest.ods'
database_of_interest = r'YourDatabase.db'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#IMPORT THE PRETRAINED BERT MODEL FOR GENERATING EMBEDDINGS:
model = AutoModel.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model.eval()

#GET THE STREAMS OF INTEREST:
streamdf = pd.read_excel(streams_of_interest)
url_list = streamdf.iloc[:,2].tolist()
name_list = streamdf.iloc[:,1].tolist()
streams_of_interest = []
for i in range(93,100+93,2):
  streams_of_interest.append(url_list[i])

#DOUBLE CHECK NAMES OF ALL COLUMNS
con = sqlite3.connect(database_of_interest)
dbcursor = con.cursor()
query = "PRAGMA TABLE_INFO(DBdb)"
dbcursor.execute(query)
print(dbcursor.fetchall())

#ADDITIONAL CHECKS OF TYPES OF AUDIENCES
query = f"SELECT DISTINCT ListofBadges FROM DBdb WHERE URL = ?"
df = pd.read_sql(query,con,params = [streams_of_interest[40]])
df


Functions for:
1. Identifying all donors and non-donors in a stream of interest and aggregating all messages an individual sent together
2. Applying (1) to all streams of interest and producing a dataframe containing this aggregated data, along with the timescale of the stream
3. Using the output of (2) and running this data through the model to generate embeddings, then converting these embeddings to a form that the transformer can use

In [ ]:
#DEFINE FUNCTION TO GO THROUGH STREAM OF INTEREST AND CONVERT IT TO A SET OF EMBEDDINGS
def message_extractor(stream):
  #GET THE LAST MESSAGE TIME
  query = f"""
  SELECT ListofTimes
  FROM DBdb
  WHERE URL = (?)
  """
  df = pd.read_sql(query, con, params=[stream])
  if not df.empty:
      df["ListofTimes"] = df["ListofTimes"].astype(float)
      last_message_time = df["ListofTimes"].max()
  else:
      last_message_time = None

  #EXTRACT AND FORMAT ALL THE MESSAGES

  query = f"""
  SELECT ListofAuthors, json_group_array(json_object('message', ListofMessages, 'time', ListofTimes)) as messages_and_times
  FROM DBdb
  WHERE URL = (?) AND ListofMessages != ''
  GROUP BY ListofAuthors
  """

  df_all = pd.read_sql(query,con,params = [stream])

  #FIND DONORS
  paid_message_types = ['paid_message','paid_sticker','sponsorships_gift_purchase_announcement']
  placeholders = ', '.join('?' for _ in paid_message_types)

  query_donors = f"""
  SELECT DISTINCT ListofAuthors
  FROM DBdb
  WHERE URL = (?)
  AND TypeofMessages IN ({placeholders})
  """

  df_donors_check = pd.read_sql(query_donors, con, params=[stream] + paid_message_types)
  donors = set(df_donors_check['ListofAuthors'])

  df_all['is_donor'] = df_all['ListofAuthors'].apply(lambda x: x in donors)
  df_donors = df_all[df_all['is_donor'] == True]
  df_non_donors = df_all[df_all['is_donor'] == False]

  return df_donors, df_non_donors, last_message_time

#DEFINE FUNCTION THAT APPLIES THIS TO ALL RELEVENT STREAMS

def data_aggregator(streams_of_interest):
  all_donors = []
  all_non_donors = []
  all_last_message_time = []
  for stream in streams_of_interest:
    df_donors, df_non_donors, last_message_time = message_extractor(stream)
    df_donors['final_time'] = last_message_time
    df_non_donors['final_time'] = last_message_time
    all_donors.append(df_donors)
    all_non_donors.append(df_non_donors)
    all_last_message_time.append(last_message_time)
    all_donors_df = pd.concat(all_donors, ignore_index=True)
    all_non_donors_df = pd.concat(all_non_donors, ignore_index=True)

  return all_donors_df, all_non_donors_df, all_last_message_time

#DEFINE FUNCTION THAT APPLIES THIS TO ALL RELEVENT STREAMS
def embedding_generator(df,max_length):
  dataset = df.copy()
  dataset['Embeddings'] = ""
  dataset['Mask'] = ""

  for i in range(len(df)):
    padding_gen = np.zeros((max_length,768+1+1))
    #The mask must also include the CLS token that will be added on later
    mask_gen = np.zeros((max_length+1))
    print(i)
    last_message_time = dataset.iloc[i]['final_time']
    message_time_output = json.loads(dataset.iloc[i]['messages_and_times'])
    individual_embeddings = []
    for entry in message_time_output:
      tokenized = tokenizer(entry['message'], padding=False, return_tensors="pt")
      with torch.no_grad():
        embedding = model(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask']).last_hidden_state.mean(dim=1).cpu().numpy()[0]
      time = entry['time']
      composite_vector = np.concatenate((embedding, [float(time)], [last_message_time]))
      individual_embeddings.append(composite_vector)
    individual_embeddings = np.vstack((individual_embeddings))
    padding_gen[:len(individual_embeddings),:] = individual_embeddings
    mask_gen[:len(individual_embeddings)+1] = 1
    dataset['Embeddings'].iloc[i] = padding_gen
    dataset[f'Mask'].iloc[i] = mask_gen
  return dataset

Variant of function 3 for scaling time

In [ ]:
#DEFINE FUNCTION THAT APPLIES THIS TO ALL RELEVENT STREAMS
def embedding_generator(df,max_length,max_time):
  dataset = df.copy()
  dataset['Embeddings'] = ""
  dataset['Mask'] = ""

  for i in range(len(df)):
    padding_gen = np.zeros((max_length,768+1+1))
    #The mask must also include the CLS token that will be added on later
    mask_gen = np.zeros((max_length+1))
    print(i)
    last_message_time = dataset.iloc[i]['final_time']
    message_time_output = json.loads(dataset.iloc[i]['messages_and_times'])
    individual_embeddings = []
    for entry in message_time_output:
      tokenized = tokenizer(entry['message'], padding=False, return_tensors="pt")
      with torch.no_grad():
        embedding = model(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask']).last_hidden_state.mean(dim=1).cpu().numpy()[0]
      time = entry['time']
      composite_vector = np.concatenate((embedding, [float(time)]/max_time, [last_message_time]/max_time))
      individual_embeddings.append(composite_vector)
    individual_embeddings = np.vstack((individual_embeddings))
    padding_gen[:len(individual_embeddings),:] = individual_embeddings
    mask_gen[:len(individual_embeddings)+1] = 1
    dataset['Embeddings'].iloc[i] = padding_gen
    dataset[f'Mask'].iloc[i] = mask_gen
  return dataset

The following code prints out general summary statistics about the dataset we are analyzing. It also generates max length - the maximum number of messages that any person sent in a stream. This is used to set the number of tokens our transformer will use:

In [ ]:
#LOOK AT BOTH THE NUMBER OF SAMPLES AND THE MAX LENGTH FOR BOTH DONORS AND NON-DONORS

paid_message_types = ['paid_message','paid_sticker','sponsorships_gift_purchase_announcement']
placeholders = ', '.join('?' for _ in paid_message_types)

#DONORS
query = f"""
SELECT ListofAuthors, COUNT(*)
FROM DBdb
WHERE URL = (?)
  AND ListofMessages != ''
  AND ListofAuthors IN (
    SELECT DISTINCT ListofAuthors
    FROM DBdb
    WHERE URL = (?)
      AND TypeofMessages IN ({placeholders})
  )
GROUP BY ListofAuthors
"""
num_donors = []
max_donor_messages = []
for stream in streams_of_interest:
  df = pd.read_sql(query,con,params = [stream,stream] + paid_message_types)
  max_donor_messages.append(df['COUNT(*)'].max())
  num_donors.append(len(df))


#NON DONORS
query = f"""
SELECT ListofAuthors, COUNT(*)
FROM DBdb
WHERE URL = (?)
  AND ListofMessages != ''
  AND ListofAuthors IN (
    SELECT DISTINCT ListofAuthors
    FROM DBdb
    WHERE URL = (?)
      AND TypeofMessages NOT IN ({placeholders})
  )
GROUP BY ListofAuthors
"""
num_nondonors = []
max_nondonor_messages = []
for stream in streams_of_interest:
  df = pd.read_sql(query,con,params = [stream,stream] + paid_message_types)
  max_nondonor_messages.append(df['COUNT(*)'].max())
  num_nondonors.append(len(df))

total_donor_samples = sum(num_donors)
total_nondonor_samples = sum(num_nondonors)
print('Total number of donor samples:{}'.format(total_donor_samples))
print('Total number of nondonor samples:{}'.format(total_nondonor_samples))
max_length = max(max_donor_messages + max_nondonor_messages)
print('Max length:{}'.format(max_length))



query = f"""
SELECT MAX(ListofTimes)
FROM DBdb
WHERE URL = (?)
"""

last_time = []
for stream in streams_of_interest:
  df = pd.read_sql(query,con,params = [stream,stream] + paid_message_types)
  last_time.append(df['COUNT(*)'].max())
  num_nondonors.append(len(df))

In [ ]:
query = f"""
SELECT ListofTimes
FROM DBdb
WHERE URL = (?)
"""

last_time = []
for stream in streams_of_interest:
  df = pd.read_sql(query,con,params = [stream,])
  if len(df['ListofTimes'].values)!= 0:
      last_time.append(max(df['ListofTimes'].values.astype(float)))
max_time = max(last_time)
max_time

Use the define functions to generate a test/training set of the desired size

In [ ]:
#GET ALL THE RELEVENT DATA FROM THE SQL DATABASES
all_donors_df, all_non_donors_df, all_last_message_time = data_aggregator(streams_of_interest)

#DEFINE THE DESIRED NUMBER OF SAMPLES
num_samples = 2500
num_train = 2000
#SAMPLE AND SPLIT INTO TEST AND TRAINING DATASETS
dataset_donor = all_donors_df.sample(n=num_samples)
dataset_donor_train = dataset_donor.iloc[:num_train]
dataset_donor_test = dataset_donor.iloc[num_train:]
dataset_non_donor = all_non_donors_df.sample(n=num_samples)
dataset_non_donor_train = dataset_non_donor.iloc[:num_train]
dataset_non_donor_test = dataset_non_donor.iloc[num_train:]

#GENERATE A DATAFRAME WITH THE TEST AND TRAINING DATA
dataset_donor_test['label'] = 1
dataset_donor_train['label'] = 1
dataset_non_donor_test['label'] = 0
dataset_non_donor_train['label'] = 0
training_dataset = pd.concat([dataset_donor_train, dataset_non_donor_train], ignore_index=True)
test_dataset = pd.concat([dataset_donor_test, dataset_non_donor_test], ignore_index=True)

#GENERATE EMBEDDINGS FOR THE TRAINING AND TEST DATASETS
embedded_training = embedding_generator(training_dataset,max_length,max_time)
embedded_test = embedding_generator(test_dataset,max_length,max_time)

In [ ]:
embedded_test.to_pickle('/content/drive/MyDrive/SQLDATABASES/testing_embeddings_0612024_scaled_time.pkl')
embedded_training.to_pickle('/content/drive/MyDrive/SQLDATABASES/training_embeddings_0612024_scaled_time.pkl')

In [ ]:
embedded_test.to_pickle('/content/drive/MyDrive/SQLDATABASES/testing_embeddings_0612024.pkl')

In [ ]:
embedded_training.to_pickle('/content/drive/MyDrive/SQLDATABASES/training_embeddings_0612024.pkl')

In [ ]:
embedded_test['Embeddings'].iloc[0]

In [ ]:
embedded_training['Embeddings'].iloc[0]